In [13]:
P.<x,y,z>=ProjectiveSpace(ZZ,2)
f=DynamicalSystem([x^2-2*y^2,y^2,z^2])
X = f.periodic_points(2,minimal=False,formal=True, return_scheme=True)
Y = f.change_ring(QQ).periodic_points(2,minimal=False,formal=True, return_scheme=True)
Y.rational_points(F=QQbar) == X.rational_points(F=QQbar)

True

In [1]:
P.<x,y,z>=ProjectiveSpace(ZZ,2)
points_source = [P(1 , 0, 0), P(0 , 1 , 0), P(0 , 0 , 1), P(1 , -1 , -1)]
points_target = [P(0 , 1 , 0), P(-2 , 0 , 1), P(0 , 0 , 1), P(1 , -1 , -1)]
P.point_transformation_matrix(points_source,points_target,normalize=True)

[ 0 -2  0]
[-2  0  0]
[ 0  1  1]

In [21]:
from itertools import count, product
n = 1
CR = f.coordinate_ring()
dom = f.domain()
PS = f.codomain().ambient_space()
R = f.base_ring()
N = PS.dimension_relative() + 1
F = f.nth_iterate_map(n)
L = [F[i]*CR.gen(j) - F[j]*CR.gen(i) for i in range(N)
    for j in range(i+1, N)]
L = [t for t in L if t != 0]
X = PS.subscheme(L)
if N > 2:
    hyperplane_at_infinity = PS.subscheme(CR.gens()[-1])
    d = f.degree()

    # if a point of period n lies on the hyperplane at infinity,
    # we must find a suitable hyperplane which contians no periodic points
    # before deforming
    FF = FractionField(R)
    if X.intersection(hyperplane_at_infinity).change_ring(FF).dimension() >= 0:
        attempted_combinations = {}
        hyperplane_found = False
        for height_bound in count(1):
            coeff_lst = ZZ.range(height_bound)
            for tup in product(coeff_lst, repeat=N):
                if list(tup) != [0]*len(PS.gens()):
                    if PS(tup) not in attempted_combinations:
                        attempted_combinations[PS(tup)] = 0
                        hyperplane = PS.subscheme(sum([tup[i]*PS.gens()[i] for i in range(N)]))
                        if X.intersection(hyperplane).change_ring(FF).dimension() < 0:
                            hyperplane_found = True
                            break
            if hyperplane_found:
                break
        source = PS.subscheme(CR.gens()[-1])
        mat = PS.hyperplane_transformation_matrix(source, hyperplane)
        new_f = f.conjugate(mat)
    else:
        new_f = f
        mat = matrix.identity(N)

    # we now deform by a parameter t
    T = R['t']
    t = T.gens()[0]
    Pt = ProjectiveSpace(N-1, R=T, names = [str(i) for i in CR.gens()])
    deformed_polys = [poly + t*Pt.gens()[-1]**d for poly in new_f.defining_polynomials()[:-1]]
    deformed_polys += [new_f.defining_polynomials()[-1]]
    f_deformed = DynamicalSystem(deformed_polys)

    # after deforming by the parameter, the preperiodic points with multiplicity
    # will seperate into different points. we can now calculate the minimal preperiodic
    # points with the parameter, and then specialize to get the formal periodic points
    ideal = f_deformed.periodic_points(n, return_scheme=True).defining_ideal()
    L = [poly.specialization({t:0}) for poly in ideal.gens()]
    subs_list = mat.inverse()*vector(CR.gens())
    subs = {}
    for i in range(len(subs_list)):
        subs[PS.gens()[i]] = subs_list[i]
    if R.is_field():
        X = PS.subscheme([poly.subs(subs) for poly in L])
    else:
        K = [poly.subs(subs) for poly in L]
        K = [poly*poly.denominator() for poly in K]
        X = PS.subscheme(K)

In [17]:
K.<a,b,c> = QQ[]
temp = a^2 + 1/2*b^2+ c^2
temp.denominator()

2

In [18]:
K.is_field()

False

In [22]:
X

Closed subscheme of Projective Space of dimension 2 over Integer Ring defined by:
  -x^2*y + x*y^2 + 2*y^3,
  x^2*y - x*y^2 - 2*y^3 - 2*y^2*z + 2*y*z^2,
  -x^2*z + 2*y^2*z + x*z^2

In [16]:
Y2 = Y.reduce().change_ring(ZZ)

In [21]:
len((X.defining_polynomials()))

19